In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize


In [3]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('/content/dataset_v2.xlsx')

# Display the first few rows of the DataFrame
display(df)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
0,00000038-3b3b-5102-be9a-7cf800000000,Nell’applicazione (ricarica conto)\nnon ho più...,Negative,5.0,"[""problemi_ricarica""]",0.0,1.0.0,2024-10-01
1,00000038-3b3b-5102-bed4-a60f00000000,mi succede svariate volte che non riesco ad ac...,Negative,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-02
2,00000038-3b3b-5102-bed5-edb100000000,"Applicazione troppo lenta, non si riesce a far...",Negative,1.0,"[""problemi_sistema""]",0.0,1.0.0,2024-10-02
3,00000038-3b3b-5102-bf2b-621400000000,"ho hype da due anni, mi è arrivata la nuova ca...",Negative,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-04
4,00000038-3b3b-5102-c000-0fc900000000,Da oggi sto riscontrando problemi ad accedere ...,Negative,1.0,"[""problemi_accesso"",""problemi_assistenza""]",0.0,1.0.0,2024-10-08
...,...,...,...,...,...,...,...,...
6145,NaN,è una vergogna pagare 3 euro al mese e avere p...,negative,NaN,"[""problemi_bonifici"",""problemi_di_accesso""]",NaN,1.0.0,NaT
6146,NaN,“Accesso biometrico non valido “ comincia a da...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6147,NaN,“Accesso biometrico non valido” è un messaggio...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6148,NaN,“Funzione” che richiedo da tempo: una sezione ...,negative,NaN,"[""problemi_di_comunicazione""]",NaN,1.0.0,NaT


TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def find_duplicates_tfidf(df, column="clean", threshold=0.90, min_df=2, max_df=0.85):
    texts = df[column].astype(str).tolist()

    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    X = vectorizer.fit_transform(texts)

    sim_matrix = cosine_similarity(X)

    rows = []
    for i in range(len(sim_matrix)):
        for j in range(i + 1, len(sim_matrix)):
            sim = sim_matrix[i, j]
            if sim >= threshold:
                rows.append({
                    "idx1": i,
                    "text1": texts[i],
                    "idx2": j,
                    "text2": texts[j],
                    "similarity": sim
                })

    return pd.DataFrame(rows)


In [ ]:
res_tfidf = find_duplicates_tfidf(df, "review", threshold=0.90)

In [ ]:
display(res_tfidf)

,idx1,text1,idx2,text2,similarity
0,5,Ottimo servizio,1008,ottimo servizio,1.000000
1,5,Ottimo servizio,1009,ottimo servizio,1.000000
2,5,Ottimo servizio,1038,servizio ottimo,1.000000
3,5,Ottimo servizio,1103,Ottimo servizio,1.000000
4,5,Ottimo servizio,1121,ottimo servizio.,1.000000
...,...,...,...,...,...
2934,5093,Non riesco più accedere all'app da una settima...,5094,Non riesco più accedere all'app da una settima...,0.994445
2935,5123,Non si apre,5136,Non si apre!!,1.000000
2936,5729,Supporto in caso di frodi inesistente. Nessun ...,5730,Supporto in caso di frodi inesistente. Nessun ...,0.983524
2937,5885,app lenta,5969,lenta,0.909710


SBERT

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

# Modello consigliato
SBERT_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

sbert = SentenceTransformer(SBERT_MODEL)

def find_duplicates_sbert(df, column="clean", threshold=0.85):
    texts = df[column].astype(str).tolist()

    embeddings = sbert.encode(texts, convert_to_tensor=True, normalize_embeddings=True)
    sim_matrix = util.cos_sim(embeddings, embeddings)

    n = len(texts)
    pairs = []

    rows = []
    for i in range(len(sim_matrix)):
        for j in range(i + 1, len(sim_matrix)):
            sim = sim_matrix[i, j]
            if sim >= threshold:
                rows.append({
                    "idx1": i,
                    "text1": texts[i],
                    "idx2": j,
                    "text2": texts[j],
                    "similarity": sim
                })

    return pd.DataFrame(rows)



model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
res_sbert = find_duplicates_sbert(df, "review", threshold=0.85)
display(res_sbert)

,idx1,text1,idx2,text2,similarity
0,1,mi succede svariate volte che non riesco ad ac...,423,Non riesco più a entrare nell’app. Dopo aver i...,tensor(0.8621)
1,1,mi succede svariate volte che non riesco ad ac...,424,Non riesco più a entrare nell’app. Dopo aver i...,tensor(0.8508)
2,5,Ottimo servizio,712,servizio eccellente,tensor(0.9689)
3,5,Ottimo servizio,713,servizio eccellente,tensor(0.9689)
4,5,Ottimo servizio,1008,ottimo servizio,tensor(0.9749)
...,...,...,...,...,...
16627,5776,Tutto bene carta e applicazione fatte bene e f...,5777,Tutto bene carta e applicazione fatte bene e f...,tensor(0.9704)
16628,5807,Un peggioramento continuo. Sono due giorni che...,5912,"da 2 giorni non riesco ad usare l' app, ogni v...",tensor(0.8524)
16629,5925,fatto investimento con piano accumulo servizio...,5926,fatto investimento con piano accumulo servizio...,tensor(0.9305)
16630,5989,non funziona la password,6003,non riesco ad entrare le password sono tutte e...,tensor(0.8785)


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


E5

In [5]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


# ============================================================
# 1) CARICA MODELLO E5
# ============================================================

MODEL_NAME = "intfloat/multilingual-e5-base"

tokenizer_e5 = AutoTokenizer.from_pretrained(MODEL_NAME)
model_e5 = AutoModel.from_pretrained(MODEL_NAME)


# ============================================================
# 2) FUNZIONE PER OTTENERE UN EMBEDDING E5
# ============================================================

def get_e5_embedding(text: str):
    """
    Restituisce embedding E5 normalizzato (L2).
    """
    formatted = "query: " + text

    tokens = tokenizer_e5(
        formatted,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    with torch.no_grad():
        output = model_e5(**tokens)
        emb = output.last_hidden_state[:, 0, :]  # CLS token
        emb = F.normalize(emb, p=2, dim=1)

    return emb.squeeze().numpy()


# ============================================================
# 3) COSINE SIMILARITY
# ============================================================

def cosine(a, b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))


# ============================================================
# 4) NEAR-DUPLICATE DETECTION
# ============================================================

def find_duplicates_e5(df, column="clean", threshold=0.90):
    """
    Identifica near-duplicates usando E5 semantic similarity.
    Restituisce una tabella con tutte le coppie duplicate.
    """
    texts = df[column].astype(str).tolist()

    # ---- Step 1: compute embeddings
    embeddings = np.vstack([get_e5_embedding(t) for t in texts])

    # ---- Step 2: compute full similarity matrix
    sim_matrix = embeddings @ embeddings.T  # perché già normalizzati

    # ---- Step 3: estrai coppie con sim ≥ threshold
    rows = []
    n = len(texts)

    for i in range(n):
        for j in range(i + 1, n):
            sim = sim_matrix[i, j]
            if sim >= threshold:
                rows.append({
                    "idx1": i,
                    "text1": texts[i],
                    "idx2": j,
                    "text2": texts[j],
                    "similarity": float(sim)
                })

    return pd.DataFrame(rows)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [6]:
res_E5   = find_duplicates_e5(df, "review", threshold=0.92)


In [7]:
display(res_E5)

,idx1,text1,idx2,text2,similarity
0,0,Nell’applicazione (ricarica conto)\nnon ho più...,47,Ultimo aggiornamento. \nÈ sparita l’opzione ri...,0.953370
1,0,Nell’applicazione (ricarica conto)\nnon ho più...,313,Al momento non è più possibile effettuare rica...,0.935603
2,0,Nell’applicazione (ricarica conto)\nnon ho più...,774,Io spero che lo mettono a posto Oggi dovevo fa...,0.923659
3,0,Nell’applicazione (ricarica conto)\nnon ho più...,1733,non capisco perché mi ha sempre fatto ricarica...,0.922664
4,0,Nell’applicazione (ricarica conto)\nnon ho più...,1963,Non mi piace il problema della ricarica in con...,0.933417
...,...,...,...,...,...
237776,6124,È più di una settimana che non riesco ad acced...,6132,È un problema grave non poter accedere all’app...,0.932235
237777,6124,È più di una settimana che non riesco ad acced...,6134,È una settimana che aspetto notizie sull’apert...,0.933856
237778,6129,È tutto il pomeriggio che non riesco a ricaric...,6132,È un problema grave non poter accedere all’app...,0.922021
237779,6129,È tutto il pomeriggio che non riesco a ricaric...,6135,È una settimana che non riesco a ricaricare la...,0.932827


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.




---
TF-IDF CHOOSED SINCE THE OTHERS REDUCE TOO MUCH THE DATASET


In [3]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('/content/dataset_v2.xlsx')

# Display the first few rows of the DataFrame
display(df)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
0,00000038-3b3b-5102-be9a-7cf800000000,Nell’applicazione (ricarica conto)\nnon ho più...,Negative,5.0,"[""problemi_ricarica""]",0.0,1.0.0,2024-10-01
1,00000038-3b3b-5102-bed4-a60f00000000,mi succede svariate volte che non riesco ad ac...,Negative,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-02
2,00000038-3b3b-5102-bed5-edb100000000,"Applicazione troppo lenta, non si riesce a far...",Negative,1.0,"[""problemi_sistema""]",0.0,1.0.0,2024-10-02
3,00000038-3b3b-5102-bf2b-621400000000,"ho hype da due anni, mi è arrivata la nuova ca...",Negative,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-04
4,00000038-3b3b-5102-c000-0fc900000000,Da oggi sto riscontrando problemi ad accedere ...,Negative,1.0,"[""problemi_accesso"",""problemi_assistenza""]",0.0,1.0.0,2024-10-08
...,...,...,...,...,...,...,...,...
6145,NaN,è una vergogna pagare 3 euro al mese e avere p...,negative,NaN,"[""problemi_bonifici"",""problemi_di_accesso""]",NaN,1.0.0,NaT
6146,NaN,“Accesso biometrico non valido “ comincia a da...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6147,NaN,“Accesso biometrico non valido” è un messaggio...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6148,NaN,“Funzione” che richiedo da tempo: una sezione ...,negative,NaN,"[""problemi_di_comunicazione""]",NaN,1.0.0,NaT


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def find_duplicates_tfidf(df, column="clean", threshold=0.90, min_df=2, max_df=0.85):
    texts = df[column].astype(str).tolist()

    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
    X = vectorizer.fit_transform(texts)

    sim_matrix = cosine_similarity(X)

    rows = []
    for i in range(len(sim_matrix)):
        for j in range(i + 1, len(sim_matrix)):
            sim = sim_matrix[i, j]
            if sim >= threshold:
                rows.append({
                    "idx1": i,
                    "text1": texts[i],
                    "idx2": j,
                    "text2": texts[j],
                    "similarity": sim
                })

    return pd.DataFrame(rows)


In [7]:
dups = find_duplicates_tfidf(df, column="review", threshold=0.90)

In [8]:
to_drop = dups["idx2"].unique().tolist()


In [9]:
df_no_duplicates = df.drop(index=to_drop).reset_index(drop=True)


In [10]:
display(df_no_duplicates)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
0,00000038-3b3b-5102-be9a-7cf800000000,Nell’applicazione (ricarica conto)\nnon ho più...,Negative,5.0,"[""problemi_ricarica""]",0.0,1.0.0,2024-10-01
1,00000038-3b3b-5102-bed4-a60f00000000,mi succede svariate volte che non riesco ad ac...,Negative,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-02
2,00000038-3b3b-5102-bed5-edb100000000,"Applicazione troppo lenta, non si riesce a far...",Negative,1.0,"[""problemi_sistema""]",0.0,1.0.0,2024-10-02
3,00000038-3b3b-5102-bf2b-621400000000,"ho hype da due anni, mi è arrivata la nuova ca...",Negative,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-04
4,00000038-3b3b-5102-c000-0fc900000000,Da oggi sto riscontrando problemi ad accedere ...,Negative,1.0,"[""problemi_accesso"",""problemi_assistenza""]",0.0,1.0.0,2024-10-08
...,...,...,...,...,...,...,...,...
5654,NaN,è una vergogna pagare 3 euro al mese e avere p...,negative,NaN,"[""problemi_bonifici"",""problemi_di_accesso""]",NaN,1.0.0,NaT
5655,NaN,“Accesso biometrico non valido “ comincia a da...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
5656,NaN,“Accesso biometrico non valido” è un messaggio...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
5657,NaN,“Funzione” che richiedo da tempo: una sezione ...,negative,NaN,"[""problemi_di_comunicazione""]",NaN,1.0.0,NaT


In [11]:
display(df)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
0,00000038-3b3b-5102-be9a-7cf800000000,Nell’applicazione (ricarica conto)\nnon ho più...,Negative,5.0,"[""problemi_ricarica""]",0.0,1.0.0,2024-10-01
1,00000038-3b3b-5102-bed4-a60f00000000,mi succede svariate volte che non riesco ad ac...,Negative,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-02
2,00000038-3b3b-5102-bed5-edb100000000,"Applicazione troppo lenta, non si riesce a far...",Negative,1.0,"[""problemi_sistema""]",0.0,1.0.0,2024-10-02
3,00000038-3b3b-5102-bf2b-621400000000,"ho hype da due anni, mi è arrivata la nuova ca...",Negative,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-04
4,00000038-3b3b-5102-c000-0fc900000000,Da oggi sto riscontrando problemi ad accedere ...,Negative,1.0,"[""problemi_accesso"",""problemi_assistenza""]",0.0,1.0.0,2024-10-08
...,...,...,...,...,...,...,...,...
6145,NaN,è una vergogna pagare 3 euro al mese e avere p...,negative,NaN,"[""problemi_bonifici"",""problemi_di_accesso""]",NaN,1.0.0,NaT
6146,NaN,“Accesso biometrico non valido “ comincia a da...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6147,NaN,“Accesso biometrico non valido” è un messaggio...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6148,NaN,“Funzione” che richiedo da tempo: una sezione ...,negative,NaN,"[""problemi_di_comunicazione""]",NaN,1.0.0,NaT
